In [45]:
from deit_token_drop_variant_i import DeiTForImageClassificationDropTokens
from transformers import DeiTForImageClassification
from transformers import DeiTImageProcessor
import yaml
from pathlib import Path
from datasets import load_dataset
import math
from inference_patchers_blockprune import optimize_model_deit


import torch
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

drop_token_sparse_args = yaml.safe_load(Path("trained-models/imagenet-1k_deit_drop_tokens_variant_i_small/epochs_11_blockPruningInfo_finalThreshold_0.5_blockSize_32_method_topK_tokenDropInfo_layerCount_3_layerType_default_keepRate_0.5_fused_True_20231227-041334/sparse_training_args.yaml").read_text())

drop_token_model = DeiTForImageClassificationDropTokens.from_pretrained(pretrained_model_name_or_path="trained-models/imagenet-1k_deit_drop_tokens_variant_i_small/epochs_11_blockPruningInfo_finalThreshold_0.5_blockSize_32_method_topK_tokenDropInfo_layerCount_3_layerType_default_keepRate_0.5_fused_True_20231227-041334/fine-pruned-MASKED")
baseline_model = DeiTForImageClassification.from_pretrained(pretrained_model_name_or_path="trained-models/deit-small-baseline-imagenet-1k-20240110-014420/best-baseline-loaded-at-end")

drop_token_config = drop_token_model.config
baseline_config = baseline_model.config

drop_token_image_processor = DeiTImageProcessor.from_pretrained(pretrained_model_name_or_path="trained-models/imagenet-1k_deit_drop_tokens_variant_i_small/epochs_11_blockPruningInfo_finalThreshold_0.5_blockSize_32_method_topK_tokenDropInfo_layerCount_3_layerType_default_keepRate_0.5_fused_True_20231227-041334/fine-pruned-MASKED")
baseline_image_processor = DeiTImageProcessor.from_pretrained(pretrained_model_name_or_path="trained-models/deit-small-baseline-imagenet-1k-20240110-014420/best-baseline-loaded-at-end")

In [ ]:
'''
mpca = MPCA(t=12,
            c=2,
            h=4,
            p_pe=8,
            frequency=300)
'''

In [ ]:
optimized_drop_token_model = optimize_model_deit(model=drop_token_model,
                                                 mode="dense")

In [ ]:
train_ds, test_ds = load_dataset("imagenet-1k", split=['train', 'validation'])   # Validation is TEST

splitter_to_train_val = train_ds.train_test_split(test_size=25000)    # 25k images from train as part of val            

train_ds = splitter_to_train_val['train']       # 1256167    instances
val_ds = splitter_to_train_val['test']          # 25000      instances

key_to_get_image = 'image'



image_mean, image_std = drop_token_image_processor.image_mean, drop_token_image_processor.image_std                 
size = drop_token_image_processor.crop_size["height"]                                             

normalize = Normalize(mean=image_mean, std=image_std)

_val_transforms = Compose(
        [
            Resize(size),
            CenterCrop(size),                                             
            ToTensor(),
            normalize,
        ]
    )

def val_transforms(examples):
    examples['pixel_values'] = [_val_transforms(image.convert("RGB")) for image in examples[key_to_get_image]]
    return examples

test_ds.set_transform(val_transforms) 

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

inputs = collate_fn([test_ds[i] for i in range(1)])

outputs = baseline_model(**inputs)